# Integration of Text and URL phishing model

An mbox file is inputted.All the html tags are removed

In [2]:
from bs4 import BeautifulSoup
import re
import codecs
import pickle
import pandas as pd
import numpy as np
html_page=codecs.open("phisheg.mbox",'r','utf-8')
soup = BeautifulSoup(html_page,features='lxml')
tags = soup.find_all('html')
text=str(tags)
cleantext = BeautifulSoup(text,"lxml").text




First text model is called to the dataframe. text_pred is the result of prediction.text_prob is the prababilty score.

In [136]:

filename = 'finalized1_text_model.sav'
loaded_model_text = pickle.load(open(filename, 'rb'))
data=[['1',cleantext]]


df = pd.DataFrame(data,columns = ['No','content'])

predictions = loaded_model_text.predict(df.content)
print(predictions)
text_pred=predictions[0]
pred_prob=loaded_model_text.predict_proba(df.content)
print(pred_prob)
x=np.shape(pred_prob)
pred_prob1=[]
for i in range(0,x[0]):
    for j in range(0,x[1]):
        pred_prob1.append(round(pred_prob[i][j],5))
text_prob=pred_prob1[1]        


[1]
[[0.01673557 0.98326443]]


All the ahref tags are found and the urls are appended to the list. Also http/https regular expression are found and the urls are appended to a list.

In [3]:
tags = soup.find_all('a')
list1= []
for t in tags:
    list1.append(t.attrs['href'])

href = []
[href.append(t.attrs['href']) for t.attrs['href'] in list1 if t.attrs['href'] not in href ] 
print(href)


['3D"http://interact.regions.com.ref14xhed.hgsrr3.net/update/ibsregions/cmserver/verify.cfm"']


In [4]:
import re 
  
def Find(string): 
    # findall() has been used  
    # with valid conditions for urls in string 
    url = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', string) 
    return url 
      
# Driver Code 
url=Find(cleantext)
print(url)
href.extend(url)

['http://interact.regions.com.www14xhed/update/ibsregions/cmserver/verify.cfm', 'http://interact.regions.com.www14xhed/update/ibsregions/cmserver/verify.cfm']


In [5]:
href

['3D"http://interact.regions.com.ref14xhed.hgsrr3.net/update/ibsregions/cmserver/verify.cfm"',
 'http://interact.regions.com.www14xhed/update/ibsregions/cmserver/verify.cfm',
 'http://interact.regions.com.www14xhed/update/ibsregions/cmserver/verify.cfm']

In [6]:
href = list(dict.fromkeys(href))

In [7]:
href

['3D"http://interact.regions.com.ref14xhed.hgsrr3.net/update/ibsregions/cmserver/verify.cfm"',
 'http://interact.regions.com.www14xhed/update/ibsregions/cmserver/verify.cfm']

The urls are given for feature extraction. the 5 features are extracted and the dataframe is created.

In [8]:
df=pd.DataFrame(data=href)
df.rename(columns={0:"URL"})
seperation_of_protocol = df[0].str.split("://",expand = True) #expand argument in the split method will give you a new column
seperation_domain_name = seperation_of_protocol[1].str.split("/",1,expand = True)
seperation_domain_name.columns=["domain_name","address"]
splitted_data = pd.concat([seperation_of_protocol[0],seperation_domain_name],axis=1)
splitted_data.columns = ['protocol','domain_name','address']


def long_url(l):
    l= str(l)
    """This function is defined in order to differntiate website based on the length of the URL"""
    if len(l) < 54:
        return 0
    elif len(l) >= 54 and len(l) <= 75:
        return 2
    return 1
	
splitted_data['long_url'] = df[0].apply(long_url) 
def have_at_symbol(l):
    """This function is used to check whether the URL contains @ symbol or not"""
    if "@" in str(l):
        return 1
    return 0
	
splitted_data['having_@_symbol'] = df[0].apply(have_at_symbol)
def redirection(l):
    """If the url has symbol(//) after protocol then such URL is to be classified as phishing """
    if "//" in str(l):
        return 1
    return 0
	
splitted_data['redirection_//_symbol'] = seperation_of_protocol[1].apply(redirection)
def prefix_suffix_seperation(l):
    if '-' in str(l):
        return 1
    return 0
splitted_data['prefix_suffix_seperation'] = seperation_domain_name['domain_name'].apply(prefix_suffix_seperation)
def sub_domains(l):
    l= str(l)
    if l.count('.') < 3:
        return 0
    elif l.count('.') == 3:
        return 2
    return 1

splitted_data['sub_domains'] = splitted_data['domain_name'].apply(sub_domains)
features = ['long_url', 'having_@_symbol', 'redirection_//_symbol','prefix_suffix_seperation','sub_domains']
X=splitted_data[features]

The classification is done by loading the model.Since there are many urls many url_pred and url_prob are there.

In [9]:
import pickle
filename = 'urlphishing.sav'
loaded_model = pickle.load(open(filename, 'rb'))
ls = loaded_model.predict(X)
print(ls)
ref = []
for i in range(0,len(ls)):
    if ls[i] == 0:
        ref.append('non-phishing')
    else:
        ref.append('phishing')
print(ref)
url_pred_prob = loaded_model.predict_proba(X)
print(url_pred_prob)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


[1 1]
['phishing', 'phishing']
[[0. 1.]
 [0. 1.]]


For finding probability score find the maximum of all scores. For finding prediction find the majority of all url's prediction(if there are 4 urls and 3 are phishing(prediction is 1) then the final prediction is one. If there is no majority then its non-phishing(2 are phishing 2 are non phishing) then result is non phishing.

In [144]:
#print(text_pred)
tup=np.shape(url_pred_prob)
n=tup[1]-1
x=url_pred_prob[:,[n]]
url_prob=float(max(x))

In [145]:
def findMajority(arr, n): 
    maxCount = 0; 
    index = -1 # sentinels 
    for i in range(n): 
        count = 0
        for j in range(n): 
          
            if(arr[i] == arr[j]): 
                count += 1
          
        # update maxCount if count of  
        # current element is greater 
        if(count > maxCount): 
          
            maxCount = count 
            index = i 
      
    # if maxCount is greater than n/2  
    # return the corresponding element  
    if (maxCount > n//2): 
        in1=arr[index]
      
    else: 
        print("No Majority Element")
        in1=0
    return in1

url_pred=findMajority(ls,len(ls))
print(url_pred)

1


Finally this algorithm is used for finding the overall output. Combining the outputs of url and text.

In [146]:
def is_phish(text_pred,url_pred,text_prob,url_prob):
    if(text_pred==1) and (url_pred==1):
        print('phishing')
    elif(text_pred==1) and (url_pred==0):
        if(text_prob>0.6):
            print("phishing")
        else:
            print("non-phishing")
    elif(text_pred==0) and (url_pred==1):
        if(url_prob>0.6):
            print("phishing")
        else:
            print("Non-phishing")
    else:
        print('non_phishing')
is_phish(text_pred,url_pred,text_prob,url_prob)

phishing


In [147]:
print(text_pred)
print(url_pred)
print(text_prob)
print(url_prob)

1
1
0.98326
1.0
